### Speed Gains Part 2: Optimizing updates for vmap

- introduce current limitation of update 
- how to manually overwrite
- test on a variant of model from cookbook 3

In [ ]:
import jax
import jax.numpy as jnp

from genjax import gen, normal, pretty
from genjax._src.core.pytree import Const

pretty()
key = jax.random.PRNGKey(0)

As we discussed in the previous cookbook entries, a main point of `update` is to be used for incremental computation: `update` performs algebraic simplifications of the logpdf-ratios computed in the weight that it returns. This is tracked through the `Diff` system.
A limitation of the current system is that if an address "x" has a tensor value, 

In [ ]:
@gen
def model(size_model: Const[int]):
    size_model = size_model.unwrap()
    x = normal(0.0, 1.0) @ "x"
    a = normal.vmap()(jnp.zeros(size_model), jnp.ones(size_model)) @ "a"
    b = normal.vmap()(jnp.zeros(size_model), jnp.ones(size_model)) @ "b"
    c = normal.vmap()(jnp.zeros(size_model), jnp.ones(size_model)) @ "c"
    obs = normal(jnp.sum(a) + jnp.sum(b) + jnp.sum(c) + x, 5.0) @ "obs"
    return obs

In [ ]:
# for idx in range(10):
#     old_z = tr.get_choices()["kernel", idx, "z"]
#     old_target_density = genjax.normal.logpdf(old_z, 0.0, 1.0)
#     request = StaticRequest({
#         "kernel": IndexRequest(jnp.array(idx), Regenerate(S.at["z"])),
#     })
#     new_tr, fwd_w, _, _ = request.edit(key, tr, ())
#     new_z = new_tr.get_choices()["kernel", idx, "z"]
#     new_target_density = genjax.normal.logpdf(new_z, 0.0, 1.0)
#     assert fwd_w == new_target_density - old_target_density